In [1]:
import gc
import time
import numpy as np
import pandas as pd
import warnings

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from project_utils import kd_utils
from importlib import reload

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
from sklearn.pipeline import make_pipeline, make_union, Pipeline
warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/yifan/anaconda3/envs/dragons_beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train=pd.read_csv('../input/train.csv', sep='\t')
test=pd.read_csv('../input/test.csv', sep='\t')

In [3]:
train['question_utc']=pd.to_datetime(train['question_utc'], unit='s')
test['question_utc']=pd.to_datetime(test['question_utc'], unit='s')
train['answer_utc']=pd.to_datetime(train['answer_utc'], unit='s')
test['answer_utc']=pd.to_datetime(test['answer_utc'], unit='s')

In [9]:
def count_words(s):
    return len(s.split())


def count_unq_words(s):
    return len(set(s.split()))


def make_features(df):
    init_col = df.columns.tolist()

    # datetime features
    df['q_dow'] = df['question_utc'].dt.dayofweek
    df['q_hour'] = df['question_utc'].dt.hour
    df['a_dow'] = df['answer_utc'].dt.dayofweek
    df['a_hour'] = df['answer_utc'].dt.hour

    df['timediff_a-q'] = (df['answer_utc'] - df['question_utc']).dt.seconds

    # length features
    df['q_len'] = df['question_text'].map(len)
    df['a_len'] = df['answer_text'].map(len)
    df['q_count_words'] = df['question_text'].map(count_words)
    df['a_count_words'] = df['answer_text'].map(count_words)
    df['q_count_unq_words'] = df['question_text'].map(count_unq_words)
    df['a_count_unq_words'] = df['answer_text'].map(count_unq_words)

    df.drop(init_col, axis=1, inplace=True)


# =============================================================================
# main
# =============================================================================

make_features(train)
make_features(test)


In [13]:
kd_utils.pickle_data('../feature_data/train_fs1.pkl', train)
kd_utils.pickle_data('../feature_data/test_fs1.pkl', test)